In [1]:
# %%
import torch, os, cv2
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import transforms
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from PIL import Image
from lipreading.model import Lipreading
from lipreading.optim_utils import CosineScheduler
import io
import sys
import dlib
from collections import deque
from skimage import transform as tf


# Constants for face and mouth ROI extraction
STD_SIZE = (256, 256)
STABLE_POINTS_IDS = [33, 36, 39, 42, 45]  # Stable facial landmarks for alignment
MOUTH_POINTS_START_IDX = 48  # Start index for mouth landmarks
MOUTH_POINTS_END_IDX = 68    # End index for mouth landmarks
CROP_WIDTH = 96   # Width of the mouth ROI
CROP_HEIGHT = 96  # Height of the mouth ROI

# Path to the mean face landmarks file from lrwar
MEAN_FACE_LANDMARKS_PATH = "mean_face_landmarks.txt"
predictor_path = "shape_predictor_68_face_landmarks.dat"

# Function to extract landmarks from a single frame
def extract_landmarks_from_frame(image, detector, predictor):
    """Extract facial landmarks from a single frame"""
    # Convert the image color to grayscale if it's not already
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
        
    # Detect the face
    rects = detector(gray, 1)
    if not rects:
        return None
    
    # Get the face with maximum area
    ref_rect = None
    max_area = 0
    for rect in rects:
        startX = rect.left()
        startY = rect.top()
        endX = rect.right()
        endY = rect.bottom()
        startX = max(0, startX)
        startY = max(0, startY)
        endX = min(endX, image.shape[1])
        endY = min(endY, image.shape[0])
        w = endX - startX
        h = endY - startY
        area = w * h
        if area > max_area:
            ref_rect = rect
            max_area = area
    
    if ref_rect is None:
        return None
    
    # Get the landmark points
    shape = predictor(gray, ref_rect)
    # Convert it to the NumPy Array
    shape_np = np.zeros((68, 2), dtype="int")
    for i in range(0, 68):
        shape_np[i] = (shape.part(i).x, shape.part(i).y)
    
    return shape_np

# Function to linearly interpolate landmarks between two frames
def linear_interpolate(landmarks, start_idx, stop_idx):
    """Linearly interpolate missing landmarks between start_idx and stop_idx"""
    start_landmarks = landmarks[start_idx]
    stop_landmarks = landmarks[stop_idx]
    delta = stop_landmarks - start_landmarks
    for idx in range(1, stop_idx - start_idx):
        landmarks[start_idx + idx] = start_landmarks + idx / float(stop_idx - start_idx) * delta
    return landmarks

# Function to interpolate landmarks for all frames
def landmarks_interpolate(landmarks):
    """Interpolate landmarks for frames where detection failed"""
    valid_frames_idx = [idx for idx, lm in enumerate(landmarks) if lm is not None]
    if not valid_frames_idx:
        return None
    
    # Interpolate between valid frames
    for idx in range(1, len(valid_frames_idx)):
        if valid_frames_idx[idx] - valid_frames_idx[idx - 1] == 1:
            continue
        else:
            landmarks = linear_interpolate(landmarks, valid_frames_idx[idx - 1], valid_frames_idx[idx])
    
    # Handle frames at the beginning or end where detection failed
    valid_frames_idx = [idx for idx, lm in enumerate(landmarks) if lm is not None]
    if valid_frames_idx:
        landmarks[:valid_frames_idx[0]] = [landmarks[valid_frames_idx[0]]] * valid_frames_idx[0]
        landmarks[valid_frames_idx[-1]:] = [landmarks[valid_frames_idx[-1]]] * (len(landmarks) - valid_frames_idx[-1])
    
    return landmarks

# Function to warp an image using similarity transform
def warp_img(src, dst, img, std_size):
    """Warp an image using similarity transform to align facial landmarks"""
    tform = tf.estimate_transform('similarity', src, dst)  # find the transformation matrix
    warped = tf.warp(img, inverse_map=tform.inverse, output_shape=std_size)  # wrap the frame image
    warped = warped * 255  # note output from wrap is double image (value range [0,1])
    warped = warped.astype('uint8')
    return warped, tform

# Function to cut a patch around the mouth region
def cut_patch(img, landmarks, height, width, threshold=5):
    """Cut a patch around the mouth region based on landmarks"""
    center_x, center_y = np.mean(landmarks, axis=0)

    if center_y - height < 0:
        center_y = height
    if center_y - height < 0 - threshold:
        raise Exception('too much bias in height')
    if center_x - width < 0:
        center_x = width
    if center_x - width < 0 - threshold:
        raise Exception('too much bias in width')

    if center_y + height > img.shape[0]:
        center_y = img.shape[0] - height
    if center_y + height > img.shape[0] + threshold:
        raise Exception('too much bias in height')
    if center_x + width > img.shape[1]:
        center_x = img.shape[1] - width
    if center_x + width > img.shape[1] + threshold:
        raise Exception('too much bias in width')

    cutted_img = np.copy(img[int(round(center_y) - round(height)): int(round(center_y) + round(height)),
                         int(round(center_x) - round(width)): int(round(center_x) + round(width))])
    return cutted_img

# Add the greedy CTC decoder functions
def greedy_ctc_decoder(logits, blank_index=0):
    """
    Greedy decoding for CTC.
    Assumes logits shape is (T, C) (log probabilities).
    Returns a list of predicted indices (for one sample).
    """
    # Convert to numpy if it's a tensor
    if isinstance(logits, torch.Tensor):
        logits = logits.detach().cpu().numpy()
    
    # Get the highest probability index at each timestep
    indices = np.argmax(logits, axis=1)  # (T,)
    
    # Remove duplicates and blanks
    filtered_indices = []
    prev_idx = -1
    for idx in indices:
        if idx != blank_index and idx != prev_idx:  # Skip blanks and duplicates
            filtered_indices.append(idx)
        prev_idx = idx
    
    return filtered_indices

def indices_to_text(indices, idx2char):
    """
    Converts a list of indices to text using the reverse vocabulary mapping.
    """
    return ''.join([idx2char.get(i, '') for i in indices])

def normalize_arabic_text(text):
    """
    Normalizes Arabic text by combining base characters with their diacritics.
    Returns a list of complete characters (base + diacritics).
    """
    chars = []
    current_char = ''
    
    diacritics = {
        '\u064B', '\u064C', '\u064D', '\u064E', '\u064F',
        '\u0650', '\u0651', '\u0652', '\u0670', '\u06E2',
        '\u0640'  # tatweel
    }
    
    for c in text:
        if c in diacritics:
            current_char += c
        else:
            if current_char:
                chars.append(current_char)
            current_char = c
    
    # Don't forget the last character
    if current_char:
        chars.append(current_char)
    
    return chars

def compute_cer(reference_indices, hypothesis_indices):
    """
    Computes Character Error Rate (CER) directly using token indices.
    Takes raw token indices from our vocabulary (class_mapping.txt) rather than Unicode text.
    
    Returns a tuple of (CER, reference_len, hypothesis_len, edit_distance)
    """
    # Use the indices directly - each index is one token in our vocabulary
    ref_tokens = reference_indices
    hyp_tokens = hypothesis_indices
    
    print(f"Debug - Reference tokens ({len(ref_tokens)} tokens): {ref_tokens}")
    print(f"Debug - Hypothesis tokens ({len(hyp_tokens)} tokens): {hyp_tokens}")
    
    m, n = len(ref_tokens), len(hyp_tokens)
    
    # Initialize the distance matrix
    dp = [[0 for _ in range(n+1)] for _ in range(m+1)]
    
    # Base cases: empty hypothesis or reference
    for i in range(m+1):
        dp[i][0] = i
    for j in range(n+1):
        dp[0][j] = j
    
    # Fill the distance matrix
    for i in range(1, m+1):
        for j in range(1, n+1):
            # If tokens match, no operation needed
            if ref_tokens[i-1] == hyp_tokens[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                # Minimum of:
                # 1. Substitution: dp[i-1][j-1] + 1
                # 2. Insertion: dp[i][j-1] + 1
                # 3. Deletion: dp[i-1][j] + 1
                dp[i][j] = min(dp[i-1][j-1] + 1,  # substitution
                              dp[i][j-1] + 1,      # insertion
                              dp[i-1][j] + 1)      # deletion
    
    edit_distance = dp[m][n]
    cer = edit_distance / max(m, 1)  # Avoid division by zero
    
    return cer, m, n, edit_distance


ModuleNotFoundError: No module named 'lipreading'

 # 2. Initialize the seed and the device

In [ ]:
# %%
# Setting the seed for reproducibility
seed = 0
def reset_seed():
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# Setting the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


 # 3. Dataset preparation

 ## 3.1. List of Classes

In [ ]:
# %%
def extract_label(file):
    label = []
    diacritics = {
        '\u064B',  # Fathatan
        '\u064C',  # Dammatan
        '\u064D',  # Kasratan
        '\u064E',  # Fatha
        '\u064F',  # Damma
        '\u0650',  # Kasra
        '\u0651',  # Shadda
        '\u0652',  # Sukun
        '\u06E2',  # Small High meem
    }

    sentence = pd.read_csv(file)
    for word in sentence.word:
        for char in word:
            if char not in diacritics:
                label.append(char)
            else:
                label[-1] += char

    return label

classes = set()
for i in os.listdir('Dataset/Csv (with Diacritics)'):
    file = 'Dataset/Csv (with Diacritics)/' + i
    label = extract_label(file)
    classes.update(label)

# Create mapping while safely handling Arabic characters
mapped_classes = {}
for i, c in enumerate(sorted(classes), 1):  
    mapped_classes[c] = i

# Print in a way that handles encoding properly
with open('class_mapping.txt', 'w', encoding='utf-8') as f:
    for char, idx in mapped_classes.items():
        f.write(f"{char}: {idx}\n")
    
# Just print count rather than the actual characters to avoid console encoding issues
print(f"Total characters in vocabulary: {len(mapped_classes)}")


 ## 3.2. Video Dataset Class

In [ ]:
# %%
# Defining the video dataset class
class VideoDataset(torch.utils.data.Dataset):
    def __init__(self, video_paths, label_paths, transform=None, use_roi_cropping=True):
        self.video_paths = video_paths
        self.label_paths = label_paths
        self.transform = transform
        self.use_roi_cropping = use_roi_cropping
        
        # Initialize face detector and landmark predictor if ROI cropping is enabled
        if self.use_roi_cropping:
            self.detector = dlib.get_frontal_face_detector()
            self.predictor = dlib.shape_predictor(predictor_path)
            self.mean_face_landmarks = np.loadtxt(MEAN_FACE_LANDMARKS_PATH)
        
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, index):
        video_path = self.video_paths[index]
        label_path = self.label_paths[index]
        
        if self.use_roi_cropping:
            frames = self.load_frames_with_roi(video_path)
        else:
            frames = self.load_frames(video_path)
            
        label = list(map(lambda x: mapped_classes[x], extract_label(label_path)))
        
        # Get the number of frames for sequence length
        input_length = torch.tensor(len(frames), dtype=torch.long)
        label_length = torch.tensor(len(label), dtype=torch.long)
        
        # Stack frames into a tensor of shape [C, T, H, W]
        if len(frames) > 0:
            # Stack the list of tensors into a single tensor
            stacked_frames = torch.stack(frames)  # Shape: [T, C, H, W]
            stacked_frames = stacked_frames.permute(1, 0, 2, 3)  # Shape: [C, T, H, W]
        else:
            # Handle empty frame list (shouldn't happen but just in case)
            stacked_frames = torch.zeros((1, 1, 112, 112))  # Single channel
        
        return stacked_frames, input_length, torch.tensor(label, dtype=torch.long), label_length
    
    def load_frames_with_roi(self, video_path):
        """Load video frames with ROI cropping around the mouth region using lrwar utilities"""
        frames = []
        cap = cv2.VideoCapture(video_path)
        
        # First pass: extract landmarks for all frames
        video_landmarks = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            # Extract landmarks using our function for single frame
            landmarks = extract_landmarks_from_frame(frame, self.detector, self.predictor)
            video_landmarks.append(landmarks)
        
        # Reset video capture for second pass
        cap.release()
        
        # Interpolate missing landmarks
        video_landmarks = landmarks_interpolate(video_landmarks)
        if video_landmarks is None or len(video_landmarks) == 0:
            print(f"Warning: No face detected in {video_path}. Falling back to regular frame loading.")
            return self.load_frames(video_path)
        
        # Second pass: process frames with landmarks
        cap = cv2.VideoCapture(video_path)
        frame_idx = 0
        
        # For smoothing landmarks
        smoothing_window = 5
        q_landmarks = deque(maxlen=smoothing_window)
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            # Convert to grayscale
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Add current landmarks to queue for smoothing
            if frame_idx < len(video_landmarks):
                q_landmarks.append(video_landmarks[frame_idx])
            
            # Process frame if we have enough landmarks for smoothing
            if len(q_landmarks) == smoothing_window:
                # Use mean of landmarks for stability
                smoothed_landmarks = np.mean(list(q_landmarks), axis=0)
                
                try:
                    # Align face using stable points
                    trans_frame, _ = warp_img(
                        smoothed_landmarks[STABLE_POINTS_IDS, :],
                        self.mean_face_landmarks[STABLE_POINTS_IDS, :],
                        gray_frame,
                        STD_SIZE
                    )
                    
                    # Cut mouth region
                    mouth_roi = cut_patch(
                        trans_frame,
                        smoothed_landmarks[MOUTH_POINTS_START_IDX:MOUTH_POINTS_END_IDX],
                        CROP_HEIGHT//2,
                        CROP_WIDTH//2
                    )
                    
                    # Resize to target size (112x112)
                    mouth_roi = cv2.resize(mouth_roi, (112, 112))
                    
                    # Convert to PIL Image and apply transforms if any
                    mouth_roi_pil = Image.fromarray(mouth_roi)
                    
                    if self.transform is not None:
                        mouth_roi_tensor = self.transform(mouth_roi_pil)
                    else:
                        mouth_roi_tensor = transforms.ToTensor()(mouth_roi_pil)
                        
                    frames.append(mouth_roi_tensor)
                    
                except Exception as e:
                    # If ROI extraction fails, use the full frame
                    print(f"ROI extraction failed: {e}. Using full frame.")
                    frame_pil = Image.fromarray(gray_frame)
                    if self.transform is not None:
                        frame_tensor = self.transform(frame_pil)
                    else:
                        frame_tensor = transforms.ToTensor()(frame_pil)
                    frames.append(frame_tensor)
            
            frame_idx += 1
        
        cap.release()
        return frames
    
    def load_frames(self, video_path):
        """Original method to load frames without ROI cropping"""
        frames = []
        video = cv2.VideoCapture(video_path)
        total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        for i in range(total_frames):
            video.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = video.read()
            if ret:
                # Convert to grayscale as the Lipreading model expects single-channel input
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                # Create PIL Image
                frame_pil = Image.fromarray(frame, 'L')  # 'L' is for grayscale
                frames.append(frame_pil)

        if self.transform is not None:
            frames = [self.transform(frame) for frame in frames] 
        
        return frames  # Return a list of frame tensors

# Defining the video transform
video_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.421], std=[0.165])  # For grayscale, single channel
])


 ## 3.2. Load & Split the dataset

In [ ]:
# %%
# Limit to 30 samples total for testing
videos_dir = "Dataset/Video"
labels_dir = "Dataset/Csv (with Diacritics)"
videos, labels = [], []
file_names = [file_name[:-4] for file_name in os.listdir(videos_dir)]
for file_name in file_names:
    videos.append(os.path.join(videos_dir, file_name + ".mp4"))
    labels.append(os.path.join(labels_dir, file_name + ".csv"))

if len(videos) > 30:  # Increased from 10 to 30
    videos = videos[:30]
    labels = labels[:30]
    
# Split the dataset into training, validation, test sets
X_temp, X_test, y_temp, y_test = train_test_split(videos, labels, test_size=0.2, random_state=seed)  # 20% test
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=seed)  # 25% of remaining for validation

print(f"Dataset sizes: Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")


 ## 3.4. DataLoaders

In [ ]:
# %%
def collate_fn(batch):
    """Pads data and labels with different lengths in the same batch
    """
    # Unpack the batch - each item is (frames, input_length, label, label_length)
    frames_list, input_lengths, labels_list, label_lengths = zip(*batch)
    
    # Get the max sequence length in this batch
    max_len = max(seq_len.item() for seq_len in input_lengths)
    
    # Get dimensions from the first item
    c, t, h, w = frames_list[0].shape  # c, t, h, w = channels, frames, height, width
    batch_size = len(frames_list)
    
    # Create a padded tensor for all sequences
    padded_frames = torch.zeros((batch_size, c, max_len, h, w))
    
    # Copy each sequence to the padded tensor
    for i, frames in enumerate(frames_list):
        seq_len = input_lengths[i].item()
        padded_frames[i, :, :seq_len, :, :] = frames[:, :seq_len, :, :]
    
    # Flatten labels for CTC loss
    labels_flat = []
    for label in labels_list:
        labels_flat.extend(label)
    labels_flat = torch.LongTensor(labels_flat)
    
    # Convert lengths to tensor
    input_lengths = torch.LongTensor(input_lengths)
    label_lengths = torch.LongTensor(label_lengths)
    
    return padded_frames, input_lengths, labels_flat, label_lengths


# Defining the video dataloaders (train, validation, test)
train_dataset = VideoDataset(X_train, y_train, transform=video_transform)
val_dataset = VideoDataset(X_val, y_val, transform=video_transform)
test_dataset = VideoDataset(X_test, y_test, transform=video_transform)
batch_size = 2  # Changed from 4 to 2 for testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, collate_fn=collate_fn)


 # 4. Model

In [ ]:
# %%
# Define a custom version of Lipreading that fixes the dimension issue
class FixedLipreading(Lipreading):
    def forward(self, x, lengths):
        B, C, T, H, W = x.size()
        # Process through frontend
        x = self.frontend(x)  # Shape: [B, frontend_nout=64, T, H/4, W/4]
        
        # Get new time dimension after frontend
        Tnew = x.shape[2]
        frontend_channels = x.shape[1]  # Should be 64
        
        # Reshape and permute for ResNet processing
        x = x.permute(0, 2, 1, 3, 4).contiguous()  # Shape: [B, T, frontend_nout, H/4, W/4]
        x = x.view(-1, frontend_channels, x.size(3), x.size(4))  # Shape: [B*T, frontend_nout, H/4, W/4]
        
        # Process through ResNet trunk
        x = self.trunk(x)  # Shape: [B*T, backend_out=512]
        
        # Reshape back to sequence form
        x = x.view(B, Tnew, -1)  # Shape: [B, T, backend_out]
        
        # Return features or process through TCN
        if not self.extract_feats:
            # DenseTCN expects input of shape (B, T, C)
            # The transpose is handled inside the DenseTCN forward method
            return self.tcn(x, lengths, B)
        
        return x

def initialize_model():
    # Define DenseTCN options tailored for character-level sequence recognition
    densetcn_options = {
        'block_config': [3, 3, 3],  # Configuration for DenseTCN blocks
        'growth_rate_set': [9, 9, 9],  # Ensure it's divisible by number of kernels (3)
        'reduced_size': 120,  # Ensure it's divisible by number of kernels (3)
        'kernel_size_set': [3, 5, 7],  # Multiple kernel sizes for different features
        'dilation_size_set': [1, 2, 4],  # Increasing dilation for longer dependencies
        'dropout': 0.2,  # Regularization
        'squeeze_excitation': True,  # Use SE for feature refinement
    }
    
    # Create the model with the appropriate number of classes (characters + blank)
    model = FixedLipreading(
        modality='video',
        hidden_dim=512,  # Match ResNet output size
        backbone_type='resnet',
        num_classes=len(mapped_classes) + 1,  # Number of characters in vocabulary + blank
        relu_type='prelu',
        densetcn_options=densetcn_options,
        extract_feats=False,  # We want predictions, not features
    )
    
    return model.to(device)

model = initialize_model()

# Build reverse mapping for decoding
idx2char = {v: k for k, v in mapped_classes.items()}
idx2char[0] = ""  # Blank token for CTC

# Defining the loss function and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


 # 5. Training and Evaluation

In [ ]:
# %%
# Training the model
def train_model():
    model.train()
    running_loss = 0.0
    ctc_loss = nn.CTCLoss(blank=0, zero_infinity=True)
    
    for batch_idx, (inputs, input_lengths, labels_flat, label_lengths) in enumerate(train_loader):
        inputs = inputs.to(device)
        input_lengths = input_lengths.to(device)
        labels_flat = labels_flat.to(device)
        label_lengths = label_lengths.to(device)
        
        optimizer.zero_grad()
        logits = model(inputs, input_lengths)
        log_probs = F.log_softmax(logits, dim=2)
        outputs_for_ctc = log_probs.transpose(0, 1)
        
        loss = ctc_loss(outputs_for_ctc, labels_flat, input_lengths, label_lengths)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Show only essential info for first sample in batch
        with torch.no_grad():
            sample_idx = 0
            sample_logits = log_probs[sample_idx]
            seq_len = input_lengths[sample_idx].item()
            valid_logits = sample_logits[:seq_len]
            pred_indices = greedy_ctc_decoder(valid_logits, blank_index=0)
            pred_text = indices_to_text(pred_indices, idx2char)
            
            start_idx = 0
            end_idx = label_lengths[sample_idx].item()
            target_indices = labels_flat[start_idx:end_idx].cpu().tolist()
            target_text = indices_to_text(target_indices, idx2char)
            
            print(f"Batch {batch_idx+1}:")
            print(f"  Pred text: {pred_text}")
            print(f"  Target text: {target_text}")
            print(f"  Loss: {loss.item():.4f}")
        
    return running_loss / (batch_idx + 1)

# Testing the model
def test_model():
    model.eval()
    ctc_loss = nn.CTCLoss(blank=0, zero_infinity=True, reduction='mean')
    
    with open("predictions_with_roi.txt", "w", encoding="utf-8-sig") as f:
        f.write("=== New Evaluation Run ===\n\n")
        f.write("Format: UTF-8 with Arabic support\n")
        f.write("Note: Lengths shown are token counts from class_mapping.txt\n\n")
        
        total_cer = 0
        total_loss = 0
        total_edit_distance = 0
        pred_lengths = []
        target_lengths = []
        
        with torch.no_grad():
            for i, (frames, input_lengths, labels_flat, label_lengths) in enumerate(test_loader):
                frames = frames.to(device)
                input_lengths = input_lengths.to(device)
                labels_flat = labels_flat.to(device)
                label_lengths = label_lengths.to(device)
                
                batch_size = frames.size(0)
                logits = model(frames, input_lengths)
                log_probs = F.log_softmax(logits, dim=2)
                log_probs_ctc = log_probs.transpose(0, 1)
                
                output_lengths = torch.full((logits.size(0),), logits.size(1), dtype=torch.long, device=device)
                if output_lengths.max() > input_lengths.min():
                    scale_factor = input_lengths.min().float() / output_lengths.max().float()
                    output_lengths = (output_lengths.float() * scale_factor).long()
                
                loss = ctc_loss(log_probs_ctc, labels_flat, output_lengths, label_lengths)
                logits_np = log_probs.cpu().detach().numpy()
                
                for b in range(batch_size):
                    batch_logits = logits_np[b]
                    pred_indices = greedy_ctc_decoder(batch_logits)
                    
                    start_idx = sum(label_lengths[:b].cpu().tolist()) if b > 0 else 0
                    end_idx = start_idx + label_lengths[b].item()
                    target_idx = labels_flat[start_idx:end_idx].cpu().numpy()
                    
                    print("Debug - Reference tokens ({} tokens): {}".format(len(target_idx), target_idx))
                    print("Debug - Hypothesis tokens ({} tokens): {}".format(len(pred_indices), pred_indices))
                    
                    pred_text = indices_to_text(pred_indices, idx2char)
                    target_text = indices_to_text(target_idx, idx2char)
                    
                    cer, ref_len, hyp_len, edit_distance = compute_cer(target_idx, pred_indices)
                    
                    total_cer += cer
                    total_loss += loss.item()
                    total_edit_distance += edit_distance
                    pred_lengths.append(hyp_len)
                    target_lengths.append(ref_len)
                    
                    f.write("--------------------------------------------------\n")
                    f.write(f"Sample {i * batch_size + b + 1}:\n")
                    f.write(f"Predicted indices: {pred_indices}\n")
                    f.write(f"Target indices: {list(target_idx)}\n")
                    f.write(f"Predicted text: {pred_text}\n")
                    f.write(f"Target text: {target_text}\n")
                    f.write(f"Lengths: pred={hyp_len} tokens, target={ref_len} tokens\n")
                    f.write(f"Edit Distance: {edit_distance}\n")
                    f.write(f"CER: {cer:.4f}\n")
                    f.write(f"CTC Loss: {loss.item():.4f}\n")
                    f.write("--------------------------------------------------\n\n")
            
            n_samples = len(test_loader.dataset)
            avg_cer = total_cer / n_samples
            avg_loss = total_loss / n_samples
            
            f.write("=== Summary Statistics ===\n")
            f.write(f"Total samples: {n_samples}\n")
            f.write(f"Average CER: {avg_cer:.4f}\n")
            f.write(f"Average Loss: {avg_loss:.4f}\n")
        
    return avg_cer, avg_loss


 # 6. Main Execution

In [ ]:
# %%
if __name__ == "__main__":
    # Defining the video transform
    video_transform = transforms.Compose([
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.421], std=[0.165])  # For grayscale, single channel
    ])
    
    # Defining the video dataloaders with ROI cropping
    train_dataset = VideoDataset(X_train, y_train, transform=video_transform, use_roi_cropping=True)
    val_dataset = VideoDataset(X_val, y_val, transform=video_transform, use_roi_cropping=True)
    test_dataset = VideoDataset(X_test, y_test, transform=video_transform, use_roi_cropping=True)
    
    batch_size = 2  # Small batch size for testing
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, collate_fn=collate_fn)
    
    # Test the model
    print("Testing model with ROI cropping...")
    avg_cer, avg_loss = test_model()
    print(f"Test CER: {avg_cer:.4f}, Test Loss: {avg_loss:.4f}")
